In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# from importlib import reload

import correlation as corr
from speckle_stat import SpeckleStatistics

In [ ]:
plt.style.use('dark_background')
#matplotlib.rcParams.keys()
plt.rcParams['figure.figsize'] = (10, 6)
plt.rc('font', size=12)
length = 8
width = 1.5
plt.rcParams['xtick.major.size'] = length
plt.rcParams['ytick.major.size'] = length
plt.rcParams['xtick.major.width'] = width
plt.rcParams['ytick.major.width'] = width

# 1) Autocorrelation

Example with Jungfrau average detector image from in-house beamtime xpp40718.

In [ ]:
run = 36
dat = np.load('./data_examples/proba_r{}.npz'.format(run))
pk = dat['pk']
kbar = dat['kbar']
Nroi = dat['Nroi']
aimg = dat['aimg']

roi = [380,480, 550,750]
img = aimg[0]
mask = np.zeros_like(img).astype(bool)
mask[roi[0]:roi[1], roi[2]:roi[3]] = True
plt.imshow(img*(1+1.5*mask), clim=(0,8), origin='lower')
plt.show()

## Correct for non-uniform illumination

In [ ]:
img_corr, mask2, bp = corr.correct_illumination(img, mask, kernel_size=50)

fig, ax = plt.subplots(nrows=2, figsize=(14,10))
ax[0].set_title('Correction factor')
ax[0].imshow(bp, origin='lower')
ax[1].set_title('Corrected average detector image')
ax[1].imshow(img_corr[0]*(1+mask2), clim=(0,10), origin='lower')
plt.show()

## Compute and fit autocorrelation

In [ ]:
A = corr.spatial_correlation_fourier(img_corr[0], mask=mask2)
A = corr.remove_central_corr(A)

fig, ax = plt.subplots(figsize=(10,8))
ax.imshow(A, clim=(0.5,2))
ax.set_title("Autocorrelation")
plt.show()

In [ ]:
corr.fit_correlation(A)

# 2) Contrast fit

Example with the data from the epix100, under a pixel mask.

What needs to be saved from the image processing for the contrast extraction:
- kave
- photon probabilities (at least k=0-3)
- size of the ROI

In [ ]:
run = 91
dat = np.load('./data_examples/proba_r{}.npz'.format(run))
pk = dat['pk']
kbar = dat['kbar']
Nroi = dat['Nroi']
aimg = dat['aimg']

plt.imshow(aimg, clim=(0,8))
plt.show()

In [ ]:
speck = SpeckleStatistics(kbar, pk, nRoi=Nroi, kavgRange=[0.01,0.4,15])

## Fit binomial distribution

In [ ]:
fig, ax = plt.subplots()
# beta, beta_err, fitRes = speck.fit_pk(0, ax=ax)
# print('{} +/- {}'.format(beta, beta_err))
beta, beta_err, fitRes = speck.fit_pk(1, ax=ax)
print('{} +/- {}'.format(beta, beta_err))
beta, beta_err, fitRes = speck.fit_pk(2, ax=ax)
print('{} +/- {}'.format(beta, beta_err))
beta, beta_err, fitRes = speck.fit_pk(3, ax=ax)
print('{} +/- {}'.format(beta, beta_err))

In [ ]:
fig, ax = plt.subplots()

beta, beta_err, fitRes = speck.fit_pk(1, ax=ax, bin_kavg=False)
print('{} +/- {}'.format(beta, beta_err))
beta, beta_err, fitRes = speck.fit_pk(2, ax=ax, bin_kavg=False)
print('{} +/- {}'.format(beta, beta_err))
# beta, beta_err, fitRes = speck.fit_pk(3, ax=ax, bin_kavg=False)
# print('{} +/- {}'.format(beta, beta_err))

## Maximum likelihood estimate

In [ ]:
fig, ax = plt.subplots()
beta, beta_err, M, MLE = speck.MLE_contrast(M=np.arange(1,5,0.05), ax=ax)

## More examples on lower contrast

In [ ]:
run = 125
dat = np.load('./data_examples/proba_r{}.npz'.format(run))
pk = dat['pk']
kbar = dat['kbar']
Nroi = dat['Nroi']
aimg = dat['aimg']

# plt.imshow(aimg, clim=(0,8))
# plt.show()

# FIT
speck = SpeckleStatistics(kbar, pk, nRoi=Nroi)
fig, ax = plt.subplots()
beta, beta_err, fitRes = speck.fit_pk(1, ax=ax)
print('{} +/- {}'.format(beta, beta_err))
beta, beta_err, fitRes = speck.fit_pk(2, ax=ax)
print('{} +/- {}'.format(beta, beta_err))
beta, beta_err, fitRes = speck.fit_pk(3, ax=ax)
print('{} +/- {}'.format(beta, beta_err))

# MLE
fig, ax = plt.subplots()
beta, beta_err, M, MLE = speck.MLE_contrast(ax=ax)

In [ ]:
run = 136
dat = np.load('./data_examples/proba_r{}.npz'.format(run))
pk = dat['pk']
kbar = dat['kbar']
Nroi = dat['Nroi']
aimg = dat['aimg']

# plt.imshow(aimg, clim=(0,8))
# plt.show()

# FIT
speck = SpeckleStatistics(kbar, pk, nRoi=Nroi)
fig, ax = plt.subplots()
beta, beta_err, fitRes = speck.fit_pk(1, ax=ax)
print('{} +/- {}'.format(beta, beta_err))
beta, beta_err, fitRes = speck.fit_pk(2, ax=ax)
print('{} +/- {}'.format(beta, beta_err))
beta, beta_err, fitRes = speck.fit_pk(3, ax=ax)
print('{} +/- {}'.format(beta, beta_err))

# MLE
fig, ax = plt.subplots()
beta, beta_err, M, MLE = speck.MLE_contrast(ax=ax)